In [27]:
import numpy as np
import pandas as pd

In [28]:
data_dir  = '../data/'

## Import Data

In [29]:
# Solar data
solar_data_file_loc = data_dir + 'processed/solar_data.csv'
solar_data = pd.read_csv(solar_data_file_loc)

# Wind data
wind_data_file_loc = data_dir + 'processed/wind_data.csv'
wind_data = pd.read_csv(wind_data_file_loc)

# Population data
pop_data_file_loc = data_dir + 'state/state_population.csv'
pop_data = pd.read_csv(pop_data_file_loc)

# State Area data
area_data_file_loc = data_dir + 'state/state_area.csv'
area_data = pd.read_csv(area_data_file_loc)

In [30]:
# State FIPS Codes
state_fips_file_loc = data_dir + 'keys/state_FIPS.csv'
state_fips_key = pd.read_csv(state_fips_file_loc)
state_fips_indicators = ['State Abbreviation', 'State Name']

## Clean Data

In [31]:
# Pop data FIPS codes
pop_data = pop_data.rename(columns = {'State': 'State Name'})
pop_data['State Name'] = pop_data['State Name'].apply(lambda x: x[1:]).str.upper()
pop_data = pop_data.merge(state_fips_key).drop(state_fips_indicators, axis = 1)

In [32]:
# Convert areas from km2 to m2
for col in area_data.columns:
    if 'Area' in col:
        area_data[col] = area_data[col].apply(lambda x: float(x)*(1000**2))

# Area data FIPS codes
area_data['State Name'] = area_data['State Name'].str.upper()
area_data = area_data.merge(state_fips_key).drop(state_fips_indicators, axis = 1)

## Merge Data

In [33]:
renewable_data = pop_data.merge(solar_data).merge(wind_data).merge(area_data)
renewable_data.head()

,Population,State FIPS,month,solar_avg_rad,NCDC Code,solar_net_gen_all,solar_net_gen_elc,elc_price_all,elc_price_com,elc_price_ind,...,wind_net_gen_all,wind_net_gen_elc,Avg_Wind_Speed,Total_Area,Land_Area,Water_Area,Water_Inland_Area,Water_Coastal_Area,Water_Great_Lakes_Area,Water_Territorial_Area
0,4860545,1,1,0.00387,1,0.0,0.0,89.4,107.9,53.6,...,0.0,0.0,4.232283,1.357670e+11,1.311710e+11,4.597000e+09,2.740000e+09,1.340000e+09,0.0,516000000.0
1,4860545,1,2,0.00431,1,0.0,0.0,90.1,108.8,52.8,...,0.0,0.0,4.527190,1.357670e+11,1.311710e+11,4.597000e+09,2.740000e+09,1.340000e+09,0.0,516000000.0
2,4860545,1,3,0.00477,1,0.0,0.0,91.0,108.4,55.5,...,0.0,0.0,4.669978,1.357670e+11,1.311710e+11,4.597000e+09,2.740000e+09,1.340000e+09,0.0,516000000.0
3,4860545,1,4,0.00506,1,0.0,0.0,92.6,110.5,58.7,...,0.0,0.0,3.731362,1.357670e+11,1.311710e+11,4.597000e+09,2.740000e+09,1.340000e+09,0.0,516000000.0
4,4860545,1,5,0.00499,1,0.0,0.0,93.2,110.2,60.0,...,0.0,0.0,3.745946,1.357670e+11,1.311710e+11,4.597000e+09,2.740000e+09,1.340000e+09,0.0,516000000.0


## Export Data

In [34]:
renewable_data.to_csv(data_dir + 'processed/renewable_data.csv', index = False)

## References

 * BEA, SQ1 Personal Income Summary: Personal Income, Population, Per Capita Personal Income. 2016. https://bea.gov/iTable/iTableHtml.cfm?reqid=70&step=30&isuri=1&7022=36&7023=0&7033=-1&7024=non-industry&7025=0&7026=xx&7027=2016&7001=336&7028=-1&7031=0&7040=-1&7083=levels&7029=36&7090=70
 * State Area Measurements and Internal Point Coordinates. https://www.census.gov/geo/reference/state-area.html